In [5]:
# 套件與初始化
import pandas as pd
import requests
import urllib.parse
import os
import warnings
warnings.filterwarnings('ignore')

UNIT_ID_LIST = {113: "蔬菜", 135: "果品"}
YEAR_CUTOFF = 2020 # 起始年份
# 以下填入需要查詢的水果(不支援模糊查詢)
VEGETABLES = ['蘋果', '桃子', '葡萄', '香蕉', '芒果', '木瓜', '鳳梨', '草莓', '藍莓', '覆盆子']

os.makedirs('蔬果資料', exist_ok=True)
print(f"共 {len(VEGETABLES)} 種蔬果，準備下載...")

共 10 種蔬果，準備下載...


In [6]:
import urllib3
import requests
import pandas as pd
import urllib.parse

# 停用 SSL 警告
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# 建立一個空的Dataframe，來存放所有資料
df_sum = pd.DataFrame()

# 抓取並儲存每一種蔬果的資料
for name in VEGETABLES:
    try:
        for id, kind in UNIT_ID_LIST.items():
            filter_str = f"年度 gt {YEAR_CUTOFF} and {kind}類別 like {name}"
            encoded = urllib.parse.quote(filter_str, safe='')
       
            url = f"https://data.moa.gov.tw/Service/OpenData/DataFileService.aspx?UnitId={id}&$top=1000&$skip=0&$filter={encoded}"
        
            # 加入 verify=False 停用 SSL 驗證
            r = requests.get(url, timeout=30, verify=False)
            data = r.json()
            
            if data:
                df = pd.DataFrame(data)
                fname = f"蔬果資料/{YEAR_CUTOFF}~2024年{name}.csv"
                if kind == "蔬菜":
                    df = df.rename(columns={"種植面積":"種植面積_公頃", "收穫面積":"收穫面積_公頃"}) # 針對蔬菜欄位的名稱要做修改
                df = df[["年度", "地區別", f"{kind}類別", "種植面積_公頃", "收穫面積_公頃"]]
                df = df.rename(columns={f"{kind}類別": "蔬果類別"}) # 統一欄位名稱
                df = df[df["蔬果類別"] == name] # 只篩選需要的名稱，避免不同水果被抓進來(ex:要抓葡萄，但葡萄柚也被抓進來)
                df_sum = pd.concat([df_sum, df]) # 合併資料表進入df_sum
                df.to_csv(fname, index=False, encoding='utf-8-sig')
                print(f"✅ 已儲存: {fname} ({len(df)} 筆)")
            else:
                print(f"⚠️ 無資料: {name}")
    except Exception as e:
        print(f"❌ 發生錯誤：{name} → {e}")

if len(df_sum.index) > 0:
    df_sum = df_sum.sort_values(by=['年度', '蔬果類別', '地區別'], ascending=False)
    df_sum.to_csv(f"蔬果資料/{YEAR_CUTOFF}~2024年★彙整★.csv", index=False, encoding='utf-8-sig')

⚠️ 無資料: 蘋果
✅ 已儲存: 蔬果資料/2020~2024年蘋果.csv (110 筆)
⚠️ 無資料: 桃子
⚠️ 無資料: 桃子
⚠️ 無資料: 葡萄
✅ 已儲存: 蔬果資料/2020~2024年葡萄.csv (110 筆)
⚠️ 無資料: 香蕉
✅ 已儲存: 蔬果資料/2020~2024年香蕉.csv (110 筆)
⚠️ 無資料: 芒果
✅ 已儲存: 蔬果資料/2020~2024年芒果.csv (110 筆)
⚠️ 無資料: 木瓜
✅ 已儲存: 蔬果資料/2020~2024年木瓜.csv (110 筆)
⚠️ 無資料: 鳳梨
✅ 已儲存: 蔬果資料/2020~2024年鳳梨.csv (110 筆)
✅ 已儲存: 蔬果資料/2020~2024年草莓.csv (110 筆)
⚠️ 無資料: 草莓
⚠️ 無資料: 藍莓
⚠️ 無資料: 藍莓
⚠️ 無資料: 覆盆子
⚠️ 無資料: 覆盆子
